In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import csv
import pickle
import time
import swifter
import multiprocessing as mp
import torch
from gensim import corpora, similarities, models

/usr/local/lib/python3.7/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
data_path = '../../data_2020/'
model_path = 'models/'
other_path = 'others/'
MIN_LEN = 8
k = 10
workers = 8
thd=13

In [3]:
train = pd.read_csv(data_path+'train_pre.csv')[['description_text_pre', 'paper_id']].fillna('')
test = pd.read_csv(data_path+'test_pre.csv')[['description_text_pre']].fillna('')
paper = pd.read_csv(data_path+'candidate_paper_pre.csv')[['paper_id', 'title_pro', 'keywords']].fillna('')
paper2id={v:i for i,v in enumerate(paper['paper_id'].values)}
id2paper={i:v for i,v in enumerate(paper['paper_id'].values)}
train_ans=[paper2id[i] for i in train['paper_id'].values]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def df2idf_thd(docfreq, totaldocs, log_base=2.0, add=0.0):
    idf = add + np.log(float(totaldocs) / docfreq) / np.log(log_base)
    return idf if idf >= idf_thd else add
def one(x):
    return (x+1e-10)/(x+1e-10)
tfidf =  models.TfidfModel.load(other_path+"train_idf.model")
idf=set()
for k,v in tqdm(tfidf.idfs.items()):
    if v>thd:
        idf.add(tfidf.id2word.id2token[k])
def filter_len(string):
    return set([word for word in string.split() if len(word) >= MIN_LEN])

def top_k(string, k=3):
    
    ret = sorted(string.split(), key=lambda x: len(x), reverse=True)[:k]
    ret = [kw for kw in ret if len(kw) >= MIN_LEN ]
    return set(ret)

train['description_text_pre'] = train['description_text_pre'].swifter.allow_dask_on_strings().apply(lambda x: filter_len(x))
test['description_text_pre'] = test['description_text_pre'].swifter.allow_dask_on_strings().apply(lambda x: filter_len(x))
paper['pseudo_keywords'] = paper['title_pro'].swifter.allow_dask_on_strings().apply(lambda x: top_k(x, k=k))
train_pairs_500number_idf=pd.read_csv(data_path+'train_pairs_1000number_idf.csv',header=None)
train = train.values
test = test.values
paper = paper.values

for i in tqdm(range(train.shape[0])):
    train[i][0]=train[i][0].intersection(idf)

for i in tqdm(range(test.shape[0])):
    test[i][0]=test[i][0].intersection(idf)    
    
for i in tqdm(range(paper.shape[0])):
    paper[i][3]=paper[i][3].intersection(idf)
    
    

In [5]:
from gensim import corpora
from scipy.sparse import csr_matrix,lil_matrix
train_keyword=[list(i[0]) for i in train]
test_keyword=[list(i[0]) for i in test]
paper_keyword=[list(i[3]) for i in paper]

dictionary = corpora.Dictionary(train_keyword+test_keyword+paper_keyword)

n_dict=len(dictionary.token2id)
n_train=len(train)
n_test=len(test)
n_paper=len(paper)
train_sparse=lil_matrix((n_train,n_dict))
test_sparse=lil_matrix((n_test,n_dict))
paper_sparse=lil_matrix((n_paper,n_dict))
for i,vec in tqdm(enumerate(train_keyword)):
    for e in vec:
        train_sparse[i,dictionary.token2id[e]]=1
        
for i,vec in tqdm(enumerate(test_keyword)):
    for e in vec:
        test_sparse[i,dictionary.token2id[e]]=1
    
for i,vec in tqdm(enumerate(paper_keyword)):
    for e in vec:
        paper_sparse[i,dictionary.token2id[e]]=1 
train_dot=train_sparse.dot(paper_sparse.T)
train_dot=train_dot.tolil()

test_dot=test_sparse.dot(paper_sparse.T)
test_dot=test_dot.tolil()

train_dot,test_dot

(<62974x838938 sparse matrix of type '<class 'numpy.float64'>'
 	with 1817964 stored elements in List of Lists format>,
 <34428x838938 sparse matrix of type '<class 'numpy.float64'>'
 	with 769219 stored elements in List of Lists format>)

In [6]:

hit=0
overlap=0
one=0
for i in tqdm(range(len(train_ans))):
    tfidf_30=[paper2id[pid] for pid in train_pairs_500number_idf.iloc[i,1:30].values]
    if (train_ans[i] in train_dot.rows[i]):
        if (train_ans[i] in tfidf_30):
            overlap+=1
        else:
            hit+=1
        if len(train_dot.rows[i])==1:
            one+=1
hit/len(train_ans),one

(0.06410582145012227, 163)

In [7]:
# dids=train_pairs_500number_idf[0].values
# with open(data_path+'train_pairs_500number_key2.csv', 'w', newline='') as f:
#     w = csv.writer(f)
#     for i in range(train_dot.rows.shape[0]):
#         w.writerow([dids[i]]+[id2paper[idx] for idx in train_dot.rows[i]])

In [8]:
test_pairs_500number_s2v=pd.read_csv(data_path+'test_pairs_1000number_s2v.csv',header=None)
dids=test_pairs_500number_s2v[0].values
with open(data_path+'test_pairs_500number_key.csv', 'w', newline='') as f:
    w = csv.writer(f)
    for i in range(test_dot.rows.shape[0]):
        w.writerow([dids[i]]+[id2paper[idx] for idx in test_dot.rows[i]])